# Building a Ratings Matrix

The purpose of this workbook is to build a user-item ratings matrix which will be used for a collaborative based recommender system.

The final output will be a ratings matrix which will be saved as a .pkl (pickle) file to be used in future workbooks.

***

In [2]:
import numpy as np
import pandas as pd
import pickle

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [4]:
business = pd.read_pickle('./Dataset/business.pkl')
review = pd.read_pickle('./Dataset/review_new.pkl')

In [6]:
business.shape
business.head(3)

(1705, 9)

,business_id,name,address,city,postal_code,latitude,longitude,stars,review_count
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,19107,39.955505,-75.155564,4.0,80
1,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,205 Race St,Philadelphia,19106,39.953949,-75.143226,4.0,245
2,ROeacJQwBeh05Rqg7F6TCg,BAP,1224 South St,Philadelphia,19147,39.943223,-75.162568,4.5,205


In [7]:
review.shape
review.head(3)

(474990, 8)

,review_id,user_id,business_id,stars,useful,funny,cool,date
0,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,2015-01-04 00:01:03
1,HME_ksGph3se7Aze5hxa-Q,kSMOJwJXuEUqzfmuFncK4A,kxX2SOes4o-D3ZQBkiMRfA,2,0,0,1,2014-07-13 17:25:47
2,EJWyA5wpdVMji1j4TwSZqQ,mqBWACmaHflW4eh_Ofp16Q,kxX2SOes4o-D3ZQBkiMRfA,5,13,6,5,2010-08-20 19:16:04


Make sure there are no records of missing star ratings in the comment form.

In [8]:
len(review) - review['stars'].isna().value_counts()

False    0
Name: stars, dtype: int64

Give each user a unique 'user_id' value. And save it as a new file.

In [9]:
user_table = review['user_id'].unique()
user_table = pd.DataFrame(user_table)
user_table = user_table.reset_index().rename(columns = {'index':'user_num',0:'user_id'})
user_table

pickle.dump(user_table, file=open('./Dataset/unique_user.pkl', 'wb+'))

,user_num,user_id
0,0,_7bHUi9Uuf5__HHc_Q8guQ
1,1,kSMOJwJXuEUqzfmuFncK4A
2,2,mqBWACmaHflW4eh_Ofp16Q
3,3,Z-xgVb4nM42943m2wbBkFw
4,4,2SEoXb6r6hPKrl9V9VzBgA
...,...,...
168671,168671,-2qfrhPeLqUfcfyZURBPmg
168672,168672,hncXq9D32g-KQKa8hiF9uQ
168673,168673,-2ZzM5wRWnYR3g0aTtLfeg
168674,168674,X-PJ2iZyw_zqhnwgyoqxyw


In [10]:
user_table.shape
user_table.head()

(168676, 2)

,user_num,user_id
0,0,_7bHUi9Uuf5__HHc_Q8guQ
1,1,kSMOJwJXuEUqzfmuFncK4A
2,2,mqBWACmaHflW4eh_Ofp16Q
3,3,Z-xgVb4nM42943m2wbBkFw
4,4,2SEoXb6r6hPKrl9V9VzBgA


In [11]:
business_table = review['business_id'].unique()
business_table = pd.DataFrame(business_table)
business_table = business_table.reset_index().rename(columns = {'index':'business_num',0:'business_id'})
business_table

pickle.dump(business_table, file=open('./Dataset/unique_business.pkl', 'wb+'))

,business_num,business_id
0,0,kxX2SOes4o-D3ZQBkiMRfA
1,1,YtSqYv1Q_pOltsVPSx54SA
2,2,eFvzHawVJofxSnD7TgbZtg
3,3,kq5Ghhh14r-eCxlVmlyd8w
4,4,oBhJuukGRqPVvYBfTkhuZA
...,...,...
1700,1700,3ut1fzbMfQ1VhFvHpeLOMw
1701,1701,5R3-eCIk4dRBtXo0A5MAzQ
1702,1702,KTgZXj6xh8aN_tLfI-YZ1Q
1703,1703,saVXla5i8TjE51S5uCaf6w


In [12]:
business_table.shape
business_table.head()

(1705, 2)

,business_num,business_id
0,0,kxX2SOes4o-D3ZQBkiMRfA
1,1,YtSqYv1Q_pOltsVPSx54SA
2,2,eFvzHawVJofxSnD7TgbZtg
3,3,kq5Ghhh14r-eCxlVmlyd8w
4,4,oBhJuukGRqPVvYBfTkhuZA


In [20]:
num_user = len(user_table)
num_business = len(business_table)

print('unique users:', num_user, '\nunique businesses:', num_business)

unique users: 168676 
unique businesses: 1705


In [21]:
%%time
ratings_array = np.full((num_user, num_business), np.nan)
ratings_array

CPU times: user 460 ms, sys: 647 ms, total: 1.11 s
Wall time: 1.28 s


array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

Sort data frame by most recent to least recent date (in order to assist with dropping duplicates)

In [22]:
review_rate = review.loc[:,['user_id','business_id','stars','date']].copy()
review_rate.sort_values(by='date', ascending = False, inplace = True)

review_rate.shape
review_rate.head(3)

(474990, 4)

,user_id,business_id,stars,date
335176,hDMM20MqoKVi4VMKamyfqQ,nIlmZLuMs0JuBRvAHSIf8Q,5,2022-01-19 19:46:34
337082,R8kA79smxR1cy22U5ZQ56w,PJjkH_VZeQwxEJvlQ-v7wA,5,2022-01-19 19:26:37
43664,NyvEE19AZsgCdnX4Q0fB-w,2SYEGA2fZLdzZQ9lmIfM5Q,5,2022-01-19 19:03:21


Only keep the most recent review.

In [23]:
review_rate.drop_duplicates(['user_id','business_id'], keep='first', inplace = True)
review_rate.shape
review_rate.head(3)

(460362, 4)

,user_id,business_id,stars,date
335176,hDMM20MqoKVi4VMKamyfqQ,nIlmZLuMs0JuBRvAHSIf8Q,5,2022-01-19 19:46:34
337082,R8kA79smxR1cy22U5ZQ56w,PJjkH_VZeQwxEJvlQ-v7wA,5,2022-01-19 19:26:37
43664,NyvEE19AZsgCdnX4Q0fB-w,2SYEGA2fZLdzZQ9lmIfM5Q,5,2022-01-19 19:03:21


In [24]:
# Lookup the unique integer based index values for each unique user_id and business_id
review_rate = pd.merge(review_rate, user_table, how = 'left')
review_rate = pd.merge(review_rate, business_table, how = 'left')
review_rate.head(3)

,user_id,business_id,stars,date,user_num,business_num
0,hDMM20MqoKVi4VMKamyfqQ,nIlmZLuMs0JuBRvAHSIf8Q,5,2022-01-19 19:46:34,134622,1094
1,R8kA79smxR1cy22U5ZQ56w,PJjkH_VZeQwxEJvlQ-v7wA,5,2022-01-19 19:26:37,19313,1065
2,NyvEE19AZsgCdnX4Q0fB-w,2SYEGA2fZLdzZQ9lmIfM5Q,5,2022-01-19 19:03:21,29285,95


Confirm that there are no missing values

In [25]:
review_rate['business_num'].isna().value_counts()
review_rate['user_num'].isna().value_counts()

False    460362
Name: business_num, dtype: int64

False    460362
Name: user_num, dtype: int64

In [26]:
# Transfer the star rating from the 'review_rate' df into the 'ratings_array' df
for row in review_rate.itertuples():
    user_index = row[5]
    business_index = row[6]
    rating_value = row[3]
    ratings_array[user_index, business_index] = rating_value

# Transform the 'ratings_array' into a data-frame
ratings_matrix = pd.DataFrame(ratings_array)
ratings_matrix

,0,1,2,3,4,5,6,7,8,9,...,1695,1696,1697,1698,1699,1700,1701,1702,1703,1704
0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168671,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168672,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168673,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168674,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
pickle.dump(ratings_matrix, file=open('./Dataset/ratings_matrix_new.pkl', 'wb+'))